In [1]:
#Gene Park
#Tuck Advisors Winter 2025 Internship Application


## In order to use this program you must sign in to your google account. 
##I must approve you as a user before you are able to use it.


##Importing the modules. Run this in terminal if you do not have these modules:
##pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import re

###  Helper Methods ###

#Get Methods
def get_header_segment_id(service, document_id):
    document = service.documents().get(documentId=document_id).execute()
    headers = document.get('headers', {})
    if headers:
        for header_id in headers:
            return header_id
    return None
def get_footer_segment_id(service, document_id):
    document = service.documents().get(documentId=document_id).execute()
    footers = document.get('footers', {})
    if footers:
        for footer_id in footers:
            return footer_id
    return None


##Identifies different kinds of markdown
def format_json_content(json_content):
    content_parts = []
    lines = json_content.split("\n")
    for line in lines:
        if line.startswith("###"):
            header_text = line.replace("###", "").strip()
            content_parts.append({'text': header_text, 'bold': True, 'fontSize': 15})
            content_parts.append({'text': "\n", 'bold': False, 'fontSize': 11})  # Insert newline and reset font size
        elif line.strip() == "":
            content_parts.append({'text': "\n", 'bold': False, 'fontSize': 11})
        else:
            parts = re.split(r'(\*\*.*?\*\*)', line.strip())
            for part in parts:
                if part.startswith("**") and part.endswith("**"):
                    content_parts.append({'text': part.replace("**", ""), 'bold': True, 'fontSize': 11})
                else:
                    content_parts.append({'text': part, 'bold': False, 'fontSize': 11})
            content_parts.append({'text': "\n", 'bold': False, 'fontSize': 11})  # Add a newline after each line
    return content_parts



##Wiring the information to the google doc
def write_to_google_doc(service, document_id, content_parts):
    requests = []
    index = 1

    for part in content_parts:
        if part['text']:
            requests.append({
                'insertText': {
                    'location': {'index': index},
                    'text': part['text']
                }
            })

            if part['text'].strip():
                requests.append({
                    'updateTextStyle': {
                        'range': {
                            'startIndex': index,
                            'endIndex': index + len(part['text'])
                        },
                        'textStyle': {
                            'bold': part['bold'],
                            'fontSize': {
                                'magnitude': part['fontSize'],
                                'unit': 'PT'
                            },
                            'weightedFontFamily': {
                                'fontFamily': 'Arial',
                                'weight': 400
                            }
                        },
                        'fields': 'bold,fontSize,weightedFontFamily'
                    }
                })

            index += len(part['text'])

    if requests:
        result = service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
        return result


In [2]:
def main():
    
    SCOPES = ['https://www.googleapis.com/auth/documents']
    
    ##Copy and paste your json data here from chatGPT:
    json_content = {
          "company_name": "ABC LLC",
          "matrix_cell": "3,4",
          "gptOutput": "Attached is the output generated regarding your company, ABC LLC, and Google Classroom. This analysis explores the potential synergies and strategic fit for a hypothetical acquisition of ABC LLC by Google Classroom.\n\nABC LLC specializes in providing career training and certification programs that offer accredited certifications across various professional fields, including healthcare, technology, business, and trades. Google Classroom, a product within the Google Workspace suite, is a widely-used online learning platform that serves educational institutions by facilitating the creation, distribution, and management of assignments in a paperless environment. Google Classroom is geared towards educators and students, from K-12 to higher education, with a focus on enhancing learning through technology.\n\n### Strategic Benefits of Acquiring ABC\n\nFirstly, acquiring ABC LLC would allow Google Classroom to significantly expand its course offerings. ABC’s focus on career training and certification aligns with Google Classroom's mission of democratizing education through accessible technology. By integrating ABC’s accredited certification programs, Google Classroom could offer a wider range of vocational and technical education options, making the platform more attractive to institutions and learners focused on career outcomes.\n\nSecondly, this acquisition would enhance Google Classroom's appeal to adult learners and professionals seeking upskilling or reskilling opportunities. ABC LLC’s courses target individuals who are seeking alternatives to traditional education routes, such as community colleges or four-year universities. Google Classroom, primarily used by educational institutions, would benefit from broadening its user base to include these career-oriented learners.\n\nLastly, by incorporating ABC LLC’s offerings, Google Classroom could deepen its market penetration in the continuing education and professional certification sectors. This would not only diversify Google Classroom's revenue streams but also solidify its position as a comprehensive provider of education technology solutions, from K-12 and higher education to professional certification and lifelong learning.\n\n### Products/Services\n\nABC LLC provides accredited certification courses across a broad spectrum of fields, including medical billing, phlebotomy, IT, and various trades. These services are complementary to Google Classroom’s current offerings, which focus on content delivery and classroom management for educators and students in traditional educational settings. By integrating ABC’s certifications into its platform, Google Classroom could extend its product portfolio to include vocational and technical training, offering end-to-end educational solutions from primary education through professional certification.\n\n### Customers\n\nABC LLC primarily targets individuals seeking certifications to advance or begin careers in various fields. These customers include professionals looking to upskill, individuals transitioning to new careers, and those bypassing traditional educational routes. Google Classroom’s current customer base includes K-12 schools, higher education institutions, and their students. While these customer segments differ, they share a common interest in educational advancement. Thus, there is an opportunity for Google Classroom to tap into ABC LLC’s customer base by offering its vocational and certification programs to students and educational institutions already using Google Classroom, thereby increasing the platform’s value proposition.\n\n### Applicable Cells of The M&A Matrix\n\n[INSERT M&A MATRIX HERE]\n\nThe acquisition of ABC LLC by Google Classroom fits into the following cells of The M&A Matrix™:\n\n**Cell 3: Complementary/Adjacent Products/Services, Same Customers**  \nThis cell is applicable because ABC LLC’s certification programs are complementary to Google Classroom’s current educational tools. Both companies serve learners seeking to improve their educational standing, although in different contexts. Integrating these services would allow Google Classroom to provide a more comprehensive educational experience, particularly for students and institutions looking for career-oriented certifications.\n\n**Cell 4: Complementary/Adjacent Products/Services, Different Customers**  \nThis cell is relevant as well because ABC LLC targets individuals and professionals seeking career certifications, while Google Classroom traditionally serves students in formal education. By acquiring ABC LLC, Google Classroom would diversify its customer base, extending its reach to adult learners and professionals, thus mitigating its reliance on the traditional educational sector.\n\n### Conclusion\n\nAcquiring ABC LLC would enable Google Classroom to enhance its educational offerings, expand into new customer segments, and strengthen its position as a leader in the education technology space. The complementary nature of the two companies' products and the opportunity to reach a broader audience make this acquisition a strategically sound move."
        }
    
    ## Checks if you are logged in or not. Uses Token.json for this.
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('docs', 'v1', credentials=creds)
    
    
    
    # Create a new Google Doc 
    document = service.documents().create(body={
        'title': 'Tuck Advisors Winter 2025 SWE Application'
    }).execute()
    
    
    ##Retrieves document ID. 
    document_id = document.get('documentId')
    image_url = 'https://drive.usercontent.google.com/download?id=1GkN5O4nORvgrnmw5a8KJtJW7vRp6V3Jw&export=view&authuser=1'

    
    
    # Add header to the new document
    header_segment_id = get_header_segment_id(service, document_id)
    if header_segment_id is None:
        requests = [{'createHeader': {'type': 'DEFAULT'}}]
        response = service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
        header_segment_id = get_header_segment_id(service, document_id)
    
    
    #Header that says "The M&A Matrix™ by Tuck Advisors"
    header_requests = [
        {
            'insertInlineImage': {
                'location': {'segmentId': header_segment_id, 'index': 0},
                'uri': image_url,
                'objectSize': {'height': {'magnitude': 100, 'unit': 'PT'}, 'width': {'magnitude': 100, 'unit': 'PT'}}
            }
        },
        {
            'updateParagraphStyle': {
                'range': {'segmentId': header_segment_id, 'startIndex': 0, 'endIndex': 1},
                'paragraphStyle': {'alignment': 'CENTER'},
                'fields': 'alignment'
            }
        },
        {
            'insertText': {
                'location': {'segmentId': header_segment_id, 'index': 1},
                'text': "\nThe M&A Matrix™ by Tuck Advisors\n\n"
            }
        },
        {
            'updateTextStyle': {
                'range': {'segmentId': header_segment_id, 'startIndex': 1, 'endIndex': 1 + len("The M&A Matrix™ by Tuck Advisors\n\n")},
                'textStyle': {'bold': True, 'fontSize': {'magnitude': 16, 'unit': 'PT'}, 'weightedFontFamily': {'fontFamily': 'Arial', 'weight': 700}},
                'fields': 'bold,fontSize,weightedFontFamily'
            }
        },
        {
            'updateParagraphStyle': {
                'range': {'segmentId': header_segment_id, 'startIndex': 1, 'endIndex': 1 + len("The M&A Matrix™ by Tuck Advisors\n\n")},
                'paragraphStyle': {'alignment': 'CENTER'},
                'fields': 'alignment'
            }
        }
    ]
    service.documents().batchUpdate(documentId=document_id, body={'requests': header_requests}).execute()

    # Format and write content to the document
    formatted_content = format_json_content(json_content["gptOutput"])
    write_to_google_doc(service, document_id, formatted_content)

    # Add footer to the new document
    footer_segment_id = get_footer_segment_id(service, document_id)
    if footer_segment_id is None:
        requests = [{'createFooter': {'type': 'DEFAULT'}}]
        response = service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
        footer_segment_id = get_footer_segment_id(service, document_id)

    
    #Footer that says "Private and confidential. Property of Tuck Advisors."
    footer_requests = [
        {
            'insertText': {
                'location': {'segmentId': footer_segment_id, 'index': 0},
                'text': "Private and confidential. Property of Tuck Advisors."
            }
        },
        {
            'updateTextStyle': {
                'range': {'segmentId': footer_segment_id, 'startIndex': 0, 'endIndex': len("Private and confidential. Property of Tuck Advisors.")},
                'textStyle': {'fontSize': {'magnitude': 11, 'unit': 'PT'}, 'weightedFontFamily': {'fontFamily': 'Arial', 'weight': 400}, 'bold': False},
                'fields': 'fontSize,weightedFontFamily,bold'
            }
        },
        {
            'updateParagraphStyle': {
                'range': {'segmentId': footer_segment_id, 'startIndex': 0, 'endIndex': len("Private and confidential. Property of Tuck Advisors.")},
                'paragraphStyle': {'alignment': 'CENTER'},
                'fields': 'alignment'
            }
        }
    ]

    service.documents().batchUpdate(documentId=document_id, body={'requests': footer_requests}).execute()
    

    print(f"Content added and formatted in the new document with ID: {document_id}")
    


    
if __name__ == '__main__':
    main()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=603825326418-3k8jq0jip48fpl43crjf7a7r19kjjceg.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A61101%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocuments&state=SWCRZ4JYdCZ7TnbJp9Tg7M1IUDp7qp&access_type=offline
Content added and formatted in the new document with ID: 1p05D2FSaPwmRmUtyxkopSdumv7tPdd9BudIU5mHM0_Q
